In [ ]:
!pip install plac

In [ ]:
from __future__ import unicode_literals, print_function
#import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

Training Data

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv(r'/content/drive/MyDrive/Jarvis Task/Training_Data_First_Name.csv')

In [ ]:
df['set']=df['set'].apply(lambda x:eval(x))

In [ ]:
TRAIN_DATA=list(df['set'])

In [ ]:
len(TRAIN_DATA)

2353279

Model

In [ ]:
model = None
#output_dir=Path("C:\\Users\\nithi\\Documents\\ner")
n_iter=2

In [ ]:
if model is not None:
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')
    print("Created blank 'en' model")

Created blank 'en' model


In [ ]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

In [ ]:
from spacy.training.example import Example

In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA[:300000]):
          example = Example.from_dict(nlp.make_doc(text), annotations)
          nlp.update([example],drop=0.5,sgd=optimizer,losses=losses)
          #print(losses)

100%|██████████| 300000/300000 [53:50<00:00, 92.86it/s]


In [ ]:
docx1=nlp('dr swastik gouda')
for token in docx1.ents:
    print(token.text,token.start_char, token.end_char,token.label_)

dr 0 2 PERSON


NAME PARSER

In [ ]:
!pip install nameparser

In [ ]:
from nameparser import HumanName

In [ ]:
import pandas as pd
from nameparser.config import Constants
import re
import string

In [ ]:
cn = Constants()
cn.titles.add('shri','shre','shree','shrimati','adv','capt','Prof','Smt','Kumari','Sriman','Sarvashri','Major','kee','kai')

In [ ]:
dh=pd.read_excel('/content/all_salutations (1).xlsx')
for i in dh['salutations']:
  cn.titles.add(i)

In [ ]:
def cleanup(input_string):
    pattern = re.compile(r'[^a-zA-Z\s]+')
    cleaned_string = re.sub(pattern, ' ', input_string)
    cleaned_string=cleaned_string.replace('  ',' ').replace('  ',' ').strip()

    m=cleaned_string.split()
    m1=m
    for i in range(len(m)):
      if len(m[i])==1:
        m1[i]='sw28'
    m1 = [x for x in m if x != 'sw28']
    s=''
    for i in m1:
      s+=i+' '
    return s.strip()

In [ ]:
HumanName(cleanup('Adv.V.A.Sooraj'),constants=cn)

<HumanName : [
	title: 'Adv' 
	first: '' 
	middle: '' 
	last: 'Sooraj' 
	suffix: ''
	nickname: ''
]>

In [ ]:
dh1=pd.read_csv('/content/3K Per AC Voter Data_CG_RAW.CSV')

In [ ]:
dh1

,Unnamed: 0,AC_NO,PART_NO,FM_NAME_EN,MOBILE_NO,AGE,GENDER,Rand
0,1,1,300,Rakesh,6260192134,19,M,1
1,7,1,266,SATOSH,9406132125,47,M,1
2,8,1,116,jamahir Singh,7587131608,22,M,1
3,11,1,217,ravipratap,8817431255,19,M,1
4,12,1,276,KARTIK,8871237424,44,M,1
...,...,...,...,...,...,...,...,...
87493,259283,90,166,Sanju,9424155364,33,F,159
87494,259285,90,199,JANKI,9770967065,18,F,159
87495,259286,90,113,NASARIN,7587457050,34,F,159
87496,259287,90,148,chandrvati,9406465443,26,F,159


In [ ]:
from tqdm.auto import tqdm

In [ ]:
#First_Name_Extractor
k=[]
first_name=[]
last_name=[]
count,count1=0,0
for i in tqdm(dh1['FM_NAME_EN']):
  if HumanName(cleanup(i),constants=cn).first == '':
    k.append(HumanName(cleanup(i),constants=cn).last)
    count+=1
    first_name.append(i)
    last_name.append(HumanName(cleanup(i),constants=cn).last)
    continue
  else:
    k.append(HumanName(cleanup(i),constants=cn).first)
    count1+=1

  0%|          | 0/87498 [00:00<?, ?it/s]

In [ ]:
dh1['First_Name']=k

In [ ]:
z=[]
for i,r in dh1.iterrows():
  if r['First_Name']=='':
    z.append(r['FM_NAME_EN'])
  else:
    z.append(r['First_Name'])

In [ ]:
dh1['F_Name_Replaced']=z

In [ ]:
dh1.to_csv('3K Per AC Voter Data_CG_RAW-First_Name_Extracted.csv')

In [ ]:
#Title Extractor
k1=[]
for i in dh1['Names']:
  k1.append(HumanName(cleanup(i),constants=cn).title)

In [ ]:
dh1['First_Name']=k

In [ ]:
dh1.to_csv('First-Name-Parser.csv',index=False)